## 18.3 Homework

Try different minimization methods in scipy on larger systems ($N$ up to 20), and show 
- 1 the average number of attempts to find the ground state
- 2 the time costs



### Optional
try to improve the code to make it run faster, analyze the most time consuming part and give your solution
    write function that get analytic gradient of LJ and put it into the hessien and run the code

In [5]:
import numpy as np
from numba import jit
import pandas as pd
from scipy.spatial.distance import cdist
import sys

def LJ(r):
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

# @jit 
def dLJdr(positions, dim = 3):
    """
    Calculate the force
    
    Obviously not very efficient
    """
    
    N_atom = int(len(pos)/dim)
    
    F = np.zeros([N_atom, dim]) 
    
    for i in range(N_atom): # loop over each atom
        center_atom = positions[3*i:(3*i)+3]
        for j in range(N_atom): # loop over each neighboring atom
            neighbor_atom = positions[3*j:(3*j)+3] 
            if not any(center_atom == neighbor_atom): # ensures that center atom isn't its own neighbour (returns true if false)
                r = np.linalg.norm(center_atom-neighbor_atom) # find the radial distance between center atom and neighbor  
                tmp =  -48/r**14 + 24/r**8 
                Fxyz =  tmp * (center_atom-neighbor_atom) # calculate the force in x,y,z directions
                F[i] += Fxyz  # for all neighbors j, sum x,y,z forces
    
    return F.flatten()

# @jit
def LJ_force(pos, dim = 3):
    N_atom = int(len(pos)/dim)
    pos = np.reshape(pos,[N_atom, dim])
    force = np.zeros([N_atom, dim])
    for i, pos0 in enumerate(pos):
        
        pos1 = pos.copy()
        pos1 = np.delete(pos1, i, 0)
        distance = cdist([pos0], pos1)
        r = pos1 - pos0
        r2 = np.power(distance, 2)
        r6 = np.power(r2, 3)
        r12 = np.power(r6, 2)
        force[i] = np.dot((48/r12-24/r6)/r2, r)

    return force.flatten()
    
# @jit
def total_energy(positions):
    """
    Calculate the total energy
    input:
    positions: 3*N array which represents the atomic positions
    output
    E: the total energy
    """
    E = 0
    N_atom = int(len(positions)/3)
        
    for i in range(N_atom-1):
        for j in range(i+1, N_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            dist = np.linalg.norm(pos1-pos2)
            E += LJ(dist)
    return E

def init_pos(N, L=5):
    return L*np.random.random_sample((N*3,))

def numeric_grad(f,r):
    eps = 1e-6
    return ((f(r+eps) - f(r))/eps)

def get_true_LJ(N):
    url = "http://doye.chem.ox.ac.uk/jon/structures/LJ/points/"+str(N)
    names = ['x', 'y', 'z']
    dataset = pd.read_csv(url, names=names, delim_whitespace=True)
    pos = dataset.values
    pos = np.reshape(pos, [N*3,1])
    return total_energy(pos)

In [6]:
# check if qiang's jacobian is close to mine for some N number of atoms
pos = init_pos(20)
print(np.isclose(dLJdr(pos),LJ_force(pos)))


[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True]


In [7]:
from scipy.optimize import minimize
import time

N = [4]#[4]#,8,12,16,20]

_method = ['BFGS',
           'CG',
           'L-BFGS-B']

threshold = 100
for N_atom in N:
    print('\n --------------------------------------')
    tot_steps = 0
    trueE = get_true_LJ(N_atom)
    for m in _method:
        calculations = []
        print('%s (%s)' %(m, N_atom))
        f_values = 0
        count = 0
        init_time = time.time()
        while(f_values - trueE > 1e-2 and count < threshold):
            pos = init_pos(N_atom)
            res = minimize(total_energy, pos, method=m, tol=1e-4, jac = LJ_force)
            f_values = res.fun
            calculations.append(f_values)
            count += 1 
            print('\r Step: {:d} value: {:.4f} Time: {:.2f} '
                   .format(count,res.fun,time.time()-init_time),flush=True,end='') 
        tot_steps += count
        
        print('\n')
    print('\n Avg # of attempts: ', round(tot_steps/3,2))
    print('\n')


 --------------------------------------
BFGS (4)
 Step: 1 value: -6.0000 Time: 0.05 

CG (4)
 Step: 1 value: -6.0000 Time: 0.06 

L-BFGS-B (4)
 Step: 14 value: -6.0000 Time: 0.11 


 Avg # of attempts:  5.33


